In [ ]:
# Define the path to the zip file and the destination directory
zip_file_path = '/content/drive/MyDrive/SoulPageIT_Assignment/Licplatesdetection_train.zip'  # Update this path to your zip file location
extract_dir = '/content/dataset/Licplatesdetection_train/'

# Create the destination directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print('Dataset unzipped successfully')

Dataset unzipped successfully


In [ ]:
# Define the path to the zip file and the destination directory
zip_file_path = '/content/drive/MyDrive/SoulPageIT_Assignment/Licplatesrecognition_train.zip'  # Update this path to your zip file location
extract_dir = '/content/dataset/Licplatesrecognition_train/'

# Create the destination directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print('Dataset unzipped successfully')

Dataset unzipped successfully


In [ ]:
# Define the path to the zip file and the destination directory
zip_file_path = '/content/drive/MyDrive/SoulPageIT_Assignment/test.zip'  # Update this path to your zip file location
extract_dir = '/content/dataset/test/'

# Create the destination directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print('Dataset unzipped successfully')

Dataset unzipped successfully


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

def load_detection_data(annotations_file, images_dir):
    try:
        # Load annotations
        df = pd.read_csv(annotations_file)

        # Prepare image paths and bounding boxes
        images = []
        bounding_boxes = []

        for idx, row in df.iterrows():
            image_path = os.path.join(images_dir, row["img_id"])
            xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']

            # Load image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error loading image: {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))  # Resize image to 224x224
            images.append(image)

            # Normalize bounding box coordinates
            height, width = image.shape[:2]
            xmin_norm = xmin / width
            ymin_norm = ymin / height
            xmax_norm = xmax / width
            ymax_norm = ymax / height
            bounding_boxes.append([xmin_norm, ymin_norm, xmax_norm, ymax_norm])

        images = np.array(images)
        bounding_boxes = np.array(bounding_boxes)

        return images, bounding_boxes
    except Exception as e:
        print(f"Error loading detection data: {e}")
        return None, None

# Example usage
detection_annotations_file = '/content/Licplatesdetection_train.csv'
detection_images_dir = '/content/dataset/Licplatesdetection_train/license_plates_detection_train'

detection_images, detection_bboxes = load_detection_data(detection_annotations_file, detection_images_dir)


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os

def load_recognition_data(annotations_file, images_dir):
    try:
        # Load annotations
        df = pd.read_csv(annotations_file)

        # Prepare plate images and texts
        plate_images = []
        plate_texts = []
        max_text_length = 0

        # Calculate the maximum length of plate texts
        for idx, row in df.iterrows():
            plate_text = row['text']
            max_text_length = max(max_text_length, len(plate_text))

        for idx, row in df.iterrows():
            plate_image_path = os.path.join(images_dir, row["img_id"])
            plate_text = row['text']

            # Load plate image
            plate_image = cv2.imread(plate_image_path, cv2.IMREAD_GRAYSCALE)
            if plate_image is None:
                print(f"Error loading image: {plate_image_path}")
                continue
            plate_image = cv2.resize(plate_image, (100, 50))  # Assuming fixed size for plate images
            plate_image = plate_image.astype('float32') / 255.0  # Normalize to [0, 1]
            plate_images.append(plate_image)

            # Process plate text (convert to categorical labels)
            plate_label = [ord(char) - ord('A') for char in plate_text.upper() if char.isalnum()]

            # Pad the sequence to the maximum length
            padded_plate_label = plate_label + [0] * (max_text_length - len(plate_label))
            plate_texts.append(padded_plate_label)

        plate_images = np.array(plate_images).reshape(-1, 50, 100, 1)  # Reshape for model input
        plate_texts = np.array(plate_texts)

        return plate_images, plate_texts
    except Exception as e:
        print(f"Error loading recognition data: {e}")
        return None, None

# Example usage
recognition_annotations_file = '/content/Licplatesrecognition_train.csv'
recognition_images_dir = '/content/dataset/Licplatesrecognition_train/license_plates_recognition_train'

recognition_images, recognition_texts = load_recognition_data(recognition_annotations_file, recognition_images_dir)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class SimpleDetectionModel(nn.Module):
    def __init__(self):
        super(SimpleDetectionModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 4)  # 4 outputs for bounding box coordinates

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the model, criterion and optimizer
detection_model = SimpleDetectionModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(detection_model.parameters(), lr=0.001)

# Prepare data
# Assuming `detection_images` and `detection_bboxes` are loaded and preprocessed correctly
detection_images = detection_images.transpose(0, 3, 1, 2)  # Transpose to (batch_size, channels, height, width)
detection_images = torch.tensor(detection_images, dtype=torch.float32)
detection_bboxes = torch.tensor(detection_bboxes, dtype=torch.float32)

# Split data into training and validation sets
train_size = int(0.8 * len(detection_images))
val_size = len(detection_images) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(TensorDataset(detection_images, detection_bboxes), [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Train the model
num_epochs = 10
detection_model.train()
for epoch in range(num_epochs):
    for images, bboxes in train_loader:
        images, bboxes = images.to(device), bboxes.to(device)
        optimizer.zero_grad()
        outputs = detection_model(images)
        loss = criterion(outputs, bboxes)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Save the detection model
torch.save(detection_model.state_dict(), 'license_plate_detection_model.pth')


Epoch [1/10], Loss: 4.2658
Epoch [2/10], Loss: 0.9662
Epoch [3/10], Loss: 0.3418
Epoch [4/10], Loss: 0.1997
Epoch [5/10], Loss: 0.3014
Epoch [6/10], Loss: 0.2777
Epoch [7/10], Loss: 0.2897
Epoch [8/10], Loss: 0.2623
Epoch [9/10], Loss: 0.2129
Epoch [10/10], Loss: 0.2315


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Dummy data for demonstration
recognition_images = torch.randn(900, 1, 50, 100)  # Example shape (900, 1, 50, 100)
recognition_texts = torch.randint(0, 36, (900,))  # Example labels (900 samples)

# Model definition with adjusted input size
class RecognitionModel(nn.Module):
    def __init__(self, num_classes=36):
        super(RecognitionModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Adjusted based on typical pooling effects
        self.fc1 = nn.Linear(128 * 6 * 12, 512)  # Adjusted size based on pool size
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Check the size after pooling and adjust accordingly
        print(x.size())  # Print size to debug

        # Adjust the view size based on actual output size after pooling
        x = x.view(x.size(0), -1)  # Flatten the tensor

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Preprocessing and Dataloaders
X_train_rec, X_val_rec, y_train_rec, y_val_rec = train_test_split(recognition_images, recognition_texts, test_size=0.2, random_state=42)

train_dataset_rec = TensorDataset(X_train_rec, y_train_rec)
val_dataset_rec = TensorDataset(X_val_rec, y_val_rec)

train_loader_rec = DataLoader(train_dataset_rec, batch_size=32, shuffle=True)
val_loader_rec = DataLoader(val_dataset_rec, batch_size=32, shuffle=False)

# Model Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
recognition_model = RecognitionModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(recognition_model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
recognition_model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader_rec:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = recognition_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader_rec)}")

# Save the model
torch.save(recognition_model.state_dict(), 'license_plate_recognition_model.pth')


torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([16, 128, 6, 12])
Epoch 1, Loss: 3.607109080190244
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size([32, 128, 6, 12])
torch.Size

In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Define a dataset class for the test images
class LicensePlateDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = sorted(os.listdir(root_dir))  # List of image file names

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_paths[idx])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        sample = {'image': image, 'img_name': self.image_paths[idx]}

        if self.transform:
            sample = self.transform(sample)

        return sample

# Define transformations if needed (e.g., resize, normalization)
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image or numpy ndarray to tensor
    # Add other transformations as needed
])

# Load test dataset
test_dataset = LicensePlateDataset(root_dir='/content/dataset/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_paths = os.listdir(root_dir)  # List all image filenames in the directory

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_paths[idx])
        image = cv2.imread(img_name)

        if image is None:
            raise ValueError(f"Failed to read image: {img_name}")

        # Convert BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Assuming you might need to resize or normalize the image here
        # image = cv2.resize(image, (target_size))
        # image = image.astype(np.float32) / 255.0

        return {'image': image, 'img_name': img_name}

# Define your test dataset
test_dataset = CustomDataset('/content/dataset/test')

# DataLoader for test dataset
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [ ]:
results = []

for batch in test_loader:
    images = batch['image']
    image_names = batch['img_name']

    for i in range(len(images)):
        image = images[i].numpy()  # Convert torch tensor to numpy array
        img_name = image_names[i]

        # Perform recognition
        recognized_text = recognize_license_plate(image)

        # Store results
        results.append({'image_name': img_name, 'recognized_text': recognized_text})

# Print or use results as needed
for result in results:
    print(f"Image: {result['image_name']}, Recognized text: {result['recognized_text']}")


ValueError: Failed to read image: /content/dataset/test/test

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define your RecognitionModel class (same as before)
class RecognitionModel(nn.Module):
    def __init__(self, num_classes=36):
        super(RecognitionModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 6 * 12, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load your trained recognition model
recognition_model = RecognitionModel()
recognition_model.load_state_dict(torch.load('license_plate_recognition_model.pth'))
recognition_model.eval()


RecognitionModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=36, bias=True)
)

In [ ]:
def recognize_license_plate(image):
    # Preprocess the image (similar to how you did during training)
    plate_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    plate_image = cv2.resize(plate_image, (100, 50)).astype('float32') / 255.0
    plate_image = torch.Tensor(plate_image).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions

    # Perform inference
    with torch.no_grad():
        text_logits = recognition_model(plate_image)
        predicted_text = ''.join([chr(x + ord('A')) for x in text_logits.argmax(dim=1).numpy()])

    return predicted_text


In [ ]:
# Assuming test_loader is defined correctly after fixing the image loading issue

results = []

for batch in test_loader:
    images = batch['image']
    image_names = batch['img_name']

    for i in range(len(images)):
        image = images[i].numpy()
        img_name = image_names[i]

        # Perform recognition
        recognized_text = recognize_license_plate(image)

        # Store results
        results.append({'image_name': img_name, 'recognized_text': recognized_text})

# Print or use results as needed
for result in results:
    print(f"Image: {result['image_name']}, Recognized text: {result['recognized_text']}")


Error loading image /content/dataset/test/test: Failed to read image: /content/dataset/test/test


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [ ]:
import torch
from your_module import DetectionModel, RecognitionModel  # Replace with actual module names

# Load detection model
detection_model = DetectionModel()
detection_model.load_state_dict(torch.load('/content/license_plate_detection_model.pth', map_location=torch.device('cpu')))  # Adjust map_location if necessary
detection_model.eval()

# Load recognition model
recognition_model = RecognitionModel()
recognition_model.load_state_dict(torch.load('/content/license_plate_recognition_model.pth', map_location=torch.device('cpu')))  # Adjust map_location if necessary
recognition_model.eval()

# Example inference function
def detect_and_recognize(image):
    # Your inference code here
    pass

# Example usage
example_image = detection_images[0]  # Replace with actual image
recognized_text = detect_and_recognize(example_image)
print(f"Recognized text: {recognized_text}")


ModuleNotFoundError: No module named 'your_module'

    img_id      text
0    0.jpg  117T3989
1    1.jpg  128T8086
2   10.jpg   94T3458
3  100.jpg  133T6719
4  101.jpg   68T5979


KeyError: 'image_name'

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Function to build license plate detection model
def build_detection_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    output_bbox = Dense(4, name='bounding_box')(x)  # Output for bounding box coordinates
    model = Model(inputs=input_layer, outputs=output_bbox)
    return model


In [ ]:
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

# Function to build license plate recognition model using BERT
def build_recognition_model(num_classes):
    # Load BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')

    # Freeze BERT layers
    for layer in bert_model.layers:
        layer.trainable = False

    # Build classification head
    input_ids = Input(shape=(128,), dtype=tf.int32)
    outputs = bert_model(input_ids)[1]
    outputs = Dense(num_classes, activation='softmax')(outputs)

    model = Model(inputs=input_ids, outputs=outputs)
    return model


In [ ]:
from sklearn.model_selection import train_test_split

# Load and preprocess detection data
detection_images, detection_bboxes = load_detection_data(detection_annotations_file, detection_images_zip)

# Split data into training and validation sets
X_train_det, X_val_det, y_train_det, y_val_det = train_test_split(detection_images, detection_bboxes, test_size=0.2, random_state=42)

# Build and compile detection model
detection_model = build_detection_model(input_shape=(224, 224, 3))
detection_model.compile(optimizer=Adam(lr=0.001), loss='mse')

# Train detection model
detection_model.fit(X_train_det, y_train_det, validation_data=(X_val_det, y_val_det), epochs=10, batch_size=32)

# Save detection model
detection_model.save('license_plate_detection_model.h5')


KeyError: 'image_name'

In [ ]:
import pandas as pd
import numpy as np
import zipfile
import cv2

# Paths to your provided files
detection_annotations_file = '/content/drive/MyDrive/SoulPageIT_Assignment/Licplatesdetection_train.csv'
detection_images_zip = '/content/drive/MyDrive/SoulPageIT_Assignment/Licplatesdetection_train.zip'

# Function to load and preprocess Training Set 1 (vehicle images with bounding boxes)
def load_detection_data(annotations_file, images_zip):
    # Load annotations
    df = pd.read_csv(annotations_file)

    # Extract images from zip file
    with zipfile.ZipFile(images_zip, 'r') as zip_ref:
        zip_ref.extractall('/tmp/Licplatesdetection_train')

    # Prepare image paths and bounding boxes
    images = []
    bounding_boxes = []

    for idx, row in df.iterrows():
        image_path = f'/tmp/Licplatesdetection_train/{row["image_name"]}'
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']

        # Load image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image)

        # Normalize bounding box coordinates
        width, height = image.shape[1], image.shape[0]
        xmin_norm = xmin / width
        ymin_norm = ymin / height
        xmax_norm = xmax / width
        ymax_norm = ymax / height
        bounding_boxes.append([xmin_norm, ymin_norm, xmax_norm, ymax_norm])

    images = np.array(images)
    bounding_boxes = np.array(bounding_boxes)

    return images, bounding_boxes


In [ ]:
# Paths to your provided files
recognition_annotations_file = '/content/drive/MyDrive/SoulPageIT_Assignment/Licplatesrecognition_train.csv'
recognition_images_zip = '/content/drive/MyDrive/SoulPageIT_Assignment/Licplatesrecognition_train.zip'

# Function to load and preprocess Training Set 2 (license plate images with text annotations)
def load_recognition_data(annotations_file, images_zip):
    # Load annotations
    df = pd.read_csv(annotations_file)

    # Extract images from zip file
    with zipfile.ZipFile(images_zip, 'r') as zip_ref:
        zip_ref.extractall('/tmp/Licplatesrecognition_train')

    # Prepare plate images and texts
    plate_images = []
    plate_texts = []

    for idx, row in df.iterrows():
        plate_image_path = f'/tmp/Licplatesrecognition_train/{row["image_name"]}'
        plate_text = row['plate_text']

        # Load plate image
        plate_image = cv2.imread(plate_image_path, cv2.IMREAD_GRAYSCALE)
        plate_image = cv2.resize(plate_image, (100, 50))  # Assuming fixed size for plate images
        plate_image = plate_image.astype('float32') / 255.0  # Normalize to [0, 1]
        plate_images.append(plate_image)

        # Process plate text (convert to categorical labels)
        plate_label = []
        for char in plate_text:
            if char.isalnum():
                if char.isdigit():
                    plate_label.append(ord(char) - ord('0'))
                else:
                    plate_label.append(ord(char.upper()) - ord('A') + 10)

        plate_texts.append(plate_label)

    plate_images = np.array(plate_images).reshape(-1, 50, 100, 1)  # Reshape for model input
    plate_texts = np.array(plate_texts)

    return plate_images, plate_texts


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Function to build license plate detection model
def build_detection_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    output_bbox = Dense(4, name='bounding_box')(x)  # Output for bounding box coordinates
    model = Model(inputs=input_layer, outputs=output_bbox)
    return model


In [ ]:
# Function to build license plate recognition model
def build_recognition_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output)
    return model


In [ ]:
from sklearn.model_selection import train_test_split

# Function to train license plate detection model
def train_detection_model(images, bounding_boxes):
    X_train, X_val, y_train, y_val = train_test_split(images, bounding_boxes, test_size=0.2, random_state=42)

    detection_model = build_detection_model(input_shape=(224, 224, 3))
    detection_model.compile(optimizer=Adam(lr=0.001), loss='mse')
    detection_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)
    detection_model.save('license_plate_detection_model.h5')


In [ ]:
# Function to train license plate recognition model
def train_recognition_model(images, labels):
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

    recognition_model = build_recognition_model(input_shape=(50, 100, 1), num_classes=36)  # Assuming 36 classes (26 letters + 10 digits)
    recognition_model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    recognition_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)
    recognition_model.save('license_plate_recognition_model.h5')


In [ ]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Function to evaluate license plate detection model
def evaluate_detection_model(model, X_val, y_val):
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    print(f"Mean Squared Error (MSE) for bounding boxes: {mse}")

    # Optionally, visualize some predictions
    for i in range(min(5, len(X_val))):
        image = X_val[i]
        true_bbox = y_val[i]
        pred_bbox = y_pred[i]

        # Visualize image with true and predicted bounding boxes
        # (Code for visualization depends on your specific implementation)


In [ ]:
# Function to evaluate license plate recognition model
def evaluate_recognition_model(model, X_val, y_val):
    loss, accuracy = model.evaluate(X_val, y_val)
    print(f"Loss: {loss}, Accuracy: {accuracy}")

    # Optionally, print and analyze classification report or confusion matrix
    # (Code for additional evaluation metrics depends on your specific implementation)


In [ ]:
import zipfile
import os
import cv2
import numpy as np

# Function to load test set images
def load_test_data(test_zip_path):
    try:
        with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
            zip_ref.extractall('/tmp/test')

        # List test image files
        test_image_files = os.listdir('/tmp/test')
        test_image_files.sort()  # Ensure sorted order for consistency

        test_images = []
        for image_file in test_image_files:
            image_path = os.path.join('/tmp/test', image_file)
            image = cv2.imread(image_path)
            if image is None:
                print(f'Failed to load image: {image_path}')
                continue

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
            test_images.append(image_rgb)

        test_images = np.array(test_images)
        return test_images, test_image_files

    except Exception as e:
        print(f'Error loading test data: {str(e)}')
        return None, None


In [ ]:
# Path to your provided test.zip file
test_zip_path = '/content/drive/MyDrive/SoulPageIT_Assignment/test.zip'

# Load test images
test_images, test_image_files = load_test_data(test_zip_path)

if test_images is not None:
    print(f'Loaded {len(test_images)} test images successfully.')
    # Continue with model loading and inference
else:
    print('Failed to load test images. Check error messages for details.')


Failed to load image: /tmp/test/test
Loaded 0 test images successfully.


OSError: No file or directory found at license_plate_detection_model.h5